In [12]:
!pip install  beautifulsoup4

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os

def scrape_turing_ml_questions():
    url = "https://www.turing.com/interview-questions/machine-learning"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Créer un dossier pour les images si nécessaire
        if not os.path.exists('images'):
            os.makedirs('images')
            
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        qa_pairs = []
        
        blocks = soup.find_all('div', class_='OrderedBlocks_block__8Fj6_')
        
        for i, block in enumerate(blocks, 1):
            try:
                # Extraire la question
                question_div = block.find('div', class_='PrismicRichText_root__bjKvj OrderedBlocks_question__ufWup')
                question = question_div.get_text(strip=True) if question_div else ""
                
                # Extraire la réponse et les images
                answer_div = block.find('div', class_='PrismicRichText_root__bjKvj OrderedBlocks_answerText__1PzUY')
                answer = answer_div.get_text(strip=True) if answer_div else ""
                
                # Rechercher les images dans la réponse
                image_urls = []
                images = answer_div.find_all('img') if answer_div else []
                
                for j, img in enumerate(images):
                    img_url = img.get('src')
                    if img_url:
                        # Nettoyer l'URL de l'image si nécessaire
                        if not img_url.startswith(('http:', 'https:')):
                            img_url = f"https://www.turing.com{img_url}"
                            
                        # Télécharger l'image
                        try:
                            img_response = requests.get(img_url, headers=headers)
                            img_response.raise_for_status()
                            
                            # Créer un nom de fichier unique
                            img_filename = f"question_{i}_image_{j + 1}.jpg"
                            img_path = os.path.join('images', img_filename)
                            
                            # Sauvegarder l'image
                            with open(img_path, 'wb') as f:
                                f.write(img_response.content)
                                
                            image_urls.append(img_path)
                            
                        except Exception as e:
                            print(f"Erreur lors du téléchargement de l'image {img_url}: {e}")
                            continue
                
                qa_pairs.append({
                    'question': question,
                    'answer': answer,
                    'images': '|'.join(image_urls) if image_urls else ''  # Joindre les chemins d'images avec un séparateur |
                })
                
            except AttributeError as e:
                print(f"Erreur lors de l'extraction d'une paire Q&A: {e}")
                continue
        
        # Créer un DataFrame
        df = pd.DataFrame(qa_pairs)
        
        # Sauvegarder en CSV
        df.to_csv('turing_ml_questions_with_images.csv', index=False, encoding='utf-8')
        
        print(f"Scraping terminé. {len(qa_pairs)} questions-réponses ont été sauvegardées.")
        print(f"Les images ont été sauvegardées dans le dossier 'images'")
        
    except requests.RequestException as e:
        print(f"Erreur lors de la requête: {e}")
    except Exception as e:
        print(f"Une erreur est survenue: {e}")

if __name__ == "__main__":
    scrape_turing_ml_questions()

Scraping terminé. 108 questions-réponses ont été sauvegardées.
Les images ont été sauvegardées dans le dossier 'images'
